In [1]:
import os
import shutil

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lightweight-gan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 62.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.0 MB/s eta 0:00:00
  Created wheel for aim-ui: filename=aim_ui-3.29.1-py3-none-any.whl size=31195131 sha256=6b221e025ca37cfeea1766955b8ab0cd5b369b77a103cb689fe8d9fc65f959d4
  Stored in director

In [5]:
import os
from PIL import Image
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt

# --- GANTI PATH INI ---
DATASET_FOLDER = "/content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang"
# ----------------------

def simple_eda(folder_path):
    print(f"Menganalisis dataset di: {folder_path} ...")

    valid_extensions = {'.jpg', '.jpeg', '.png', '.webp', '.bmp'}
    files = [f for f in os.listdir(folder_path) if os.path.splitext(f)[1].lower() in valid_extensions]

    if not files:
        print("Folder kosong atau tidak ada gambar!")
        return

    modes = []
    sizes = []
    corrupt_files = []

    for filename in tqdm(files):
        filepath = os.path.join(folder_path, filename)
        try:
            with Image.open(filepath) as img:
                # Cek Mode Warna (RGB, L, RGBA, CMYK, dll)
                modes.append(img.mode)
                # Cek Ukuran
                sizes.append(img.size) # (width, height)
        except Exception:
            corrupt_files.append(filename)

    # --- LAPORAN ---
    print("\n" + "="*30)
    print("       HASIL ANALISIS")
    print("="*30)
    print(f"Total Gambar Terbaca: {len(modes)}")
    print(f"File Rusak/Corrupt  : {len(corrupt_files)} (Hapus file ini!)")

    print("\n--- 1. Distribusi Channel Warna ---")
    mode_counts = Counter(modes)
    for mode, count in mode_counts.items():
        status = "AMAN" if mode == "RGB" else "BAHAYA (Perlu Convert)"
        print(f"  - {mode:<5} : {count:<5} file -> {status}")

    print("\n--- 2. Distribusi Ukuran (Resolusi) ---")
    widths = [s[0] for s in sizes]
    heights = [s[1] for s in sizes]

    print(f"  - Terkecil : {min(widths)}x{min(heights)} px")
    print(f"  - Terbesar : {max(widths)}x{max(heights)} px")
    print(f"  - Rata-rata: {int(sum(widths)/len(widths))}x{int(sum(heights)/len(heights))} px")

    # Cek Gambar Kekecilan
    too_small = sum(1 for w, h in sizes if w < 256 or h < 256)
    if too_small > 0:
        print(f"\n[!] PERINGATAN: Ada {too_small} gambar di bawah 256px. Hasil mungkin buram.")
    else:
        print("\n[OK] Semua gambar resolusi cukup besar.")

    if corrupt_files:
        print("\n[!] DAFTAR FILE RUSAK:")
        print(corrupt_files)

simple_eda(DATASET_FOLDER)

Menganalisis dataset di: /content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang ...


100%|██████████| 150/150 [00:02<00:00, 52.09it/s] 


       HASIL ANALISIS
Total Gambar Terbaca: 150
File Rusak/Corrupt  : 0 (Hapus file ini!)

--- 1. Distribusi Channel Warna ---
  - RGB   : 150   file -> AMAN

--- 2. Distribusi Ukuran (Resolusi) ---
  - Terkecil : 256x256 px
  - Terbesar : 256x256 px
  - Rata-rata: 256x256 px

[OK] Semua gambar resolusi cukup besar.


In [ ]:
!lightweight_gan \
    --data "/content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang" \
    --name "batik_parang" \
    --results_dir "/content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/results" \
    --models_dir "/content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/models" \
    --load-from 5 \
    --image-size 256 \
    --batch-size 8 \
    --gradient-accumulate-every 2 \
    --disc-output-size 5 \
    --num-train-steps 20000 \
    --save-every 2500 \
    --evaluate-every 1000 \
    --amp \
    --aug-prob 0.20 \
    --aug-types "[translation,color,offset]" \
    --calculate-fid-every 5000 \
    --calculate-fid-num-images 150

loading from version 1.2.1
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang>:  63% 12550/20000 [01:13<3:03:09,  1.48s/it]G: 0.54 | D: 1.48 | GP: 0.80 | SS: 0.01
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang>:  63% 12600/20000 [02:08<2:38:28,  1.28s/it]G: 0.37 | D: 1.17 | GP: 1.01 | SS: 0.01
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang>:  63% 12650/20000 [03:11<2:36:15,  1.28s/it]G: 0.72 | D: 1.22 | GP: 1.03 | SS: 0.01
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang>:  64% 12700/20000 [04:05<2:29:17,  1.23s/it]G: 0.72 | D: 2.14 | GP: 1.05 | SS: 0.03
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GENERATOR/dataset/dataset_256/batik_parang>:  64% 12750/20000 [05:09<2:29:28,  1.24s/it]G: 0.58 | D: 1.19 | GP: 1.01 | SS: 0.01
batik_parang</content/drive/MyDrive/FP AI LANJUT/BATIK_GEN